# LINE Messaging API

# Global vars

In [3]:
PLANT_NAME = "DEFAULT NAME" # we need to find a way to keep this in the memory.
#Since the code is always running and only sends messages at some intervals, it shouldn't be that hard to do

## Broadcast Message

In [4]:
import requests
import os
from typing import Dict, Any, Optional, List

class PlantitaBot:
    def __init__(self, channel_access_token: str):
        """Initialize the Plantita LINE bot

        Args:
            channel_access_token (str): Your LINE channel access token
        """
        self.channel_access_token = channel_access_token
        self.headers = {
            'Authorization': f'Bearer {channel_access_token}',
            'Content-Type': 'application/json'
        }
        self.base_url = 'https://api.line.me/v2/bot'

    def get_followers(self) -> List[str]:
        """Get list of user IDs who are friends with your LINE Official Account

        Returns:
            List[str]: List of user IDs
        """
        user_ids = []
        next_cursor = None

        while True:
            endpoint = f'{self.base_url}/followers/ids'
            if next_cursor:
                endpoint += f'?start={next_cursor}'

            response = requests.get(endpoint, headers=self.headers)
            data = response.json()

            if 'userIds' in data:
                user_ids.extend(data['userIds'])

            if 'next' not in data:
                break

            next_cursor = data['next']

        return user_ids

    def broadcast_message(self, message: str) -> Dict[str, Any]:
        """Broadcast a message to all friends of your LINE Official Account

        Args:
            message (str): The message to broadcast

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/broadcast'

        data = {
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()




    def broadcast_plant_status(self,
                               temperature: float,
                               humidity: float,
                               light_level: float,
                               status: str = "normal") -> Dict[str, Any]:
        """Broadcast plant status to all friends

        Args:
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.broadcast_message(message)

    def send_message(self, user_id: str, message: str) -> Dict[str, Any]:
        """Send a message to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            message (str): The message to send

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        endpoint = f'{self.base_url}/message/push'

        data = {
            'to': user_id,
            'messages': [{
                'type': 'text',
                'text': message
            }]
        }

        response = requests.post(endpoint, headers=self.headers, json=data)
        return response.json()

    def send_plant_status(self,
                          user_id: str,
                          temperature: float,
                          humidity: float,
                          light_level: float,
                          status: str = "normal") -> Dict[str, Any]:
        """Send plant status to a specific user

        Args:
            user_id (str): The LINE user ID to send the message to
            temperature (float): Current temperature reading
            humidity (float): Current humidity reading
            light_level (float): Current light level reading
            status (str): Overall status of the plant ("normal", "warning", "critical")

        Returns:
            Dict[str, Any]: Response from the LINE API
        """
        message = (
            f"🌿 Plantita Status Update 🌿\n\n"
            f"Temperature: {temperature}°C\n"
            f"Humidity: {humidity}%\n"
            f"Light Level: {light_level}%\n\n"
            f"Status: {status.upper()}"
        )

        if status.lower() != "normal":
            message += "\n\n⚠️ Action needed! Please check your plant."

        return self.send_message(user_id, message)

## Example Usage of Broadcast

## Example Usage of Direct Message
**Still Not Working** Need to setup Webhook Server.

In [ ]:
# Initialize the bot
bot = PlantitaBot("HYIfkCHmidAfgMc2KcXBLUXzpNvsvJ/t5k8VwU2+ekWovndds7h/5h2Qq+SZkoYjNCPuLg0BHANjAnX81xjbfT7imjJ1s+gYZf6XU4ttM4kuFTkChGSHDZUVKzFpTgesDxTWmILCYw0vgPz/bse7TQdB04t89/1O/w1cDnyilFU=")



{'message': "The property, 'to', in the request body is invalid (line: -, column: -)"}

## GROQ Implementation

# Install Dependency

# Demonstration

In [5]:
import os

from groq import Groq

client = Groq(
    api_key="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",
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": '''You are Plantita, an expert plant care advisor with a warm, caring personality like a concerned aunt.
Current readings for Monstera Deliciosa:
        temperature=27.5,
        humidity=55,
        light_level=40,

Ideal conditions for this plant:
- Temperature: 20°C to 25°C
- Humidity: 60% to 80%
- Light Level: 50% to 80%

Please analyze these conditions and provide:
1. A caring, conversational assessment of the plant's current environment
2. Specific recommendations for improvement if needed
3. Any potential risks to the plant's health based on these conditions
4. A simple action plan for the plant owner

Keep your response friendly and encouraging, like a knowledgeable aunt giving advice about their beloved plants.
Limit your response to fit a notification on a messaging app, focus only on what the user needed to do and keep it to three to 5 sentences''',
        }
    ],
    model="llama3-8b-8192",
)

print(chat_completion.choices[0].message.content)

Sweetie, let's take a look at your Monstera Deliciosa's environment! It's doing okay, but we can make some adjustments to bring it to its full glory. Since it's a bit warm (27.5°C), I'd recommend keeping it a bit cooler, ideally between 20°C to 25°C. As for humidity, it's a bit low, but don't worry, just increase the humidity around it by 5-10% to get it to that 60-80% range. 

To avoid any potential risks, keep an eye out for pests like spider mites or mealybugs, especially if it gets hotter than expected. For now, give your plant some TLC and make sure it's getting the right amount of light – 50-80% is perfect!

Action plan: Try to keep the temperature between 20-25°C and humidity at 60-80%. Monitor for pests and adjust the light level as needed.


# Integrate sensor information with Groq

# Simulate Arduino data on PC if we forgot to carry the Arduino

# Actual arduino data

In [ ]:
#!pip install bleak


In [8]:
from bleak import discover
from bleak import BleakClient
import asyncio

async def debug_ble():
    devices = await discover()
    print("Found devices:")
    for device in devices:
        print(device)

# Run this to check available devices
loop = asyncio.get_event_loop()
loop.run_until_complete(debug_ble())


RuntimeError: This event loop is already running

In [ ]:
import asyncio
from bleak import BleakClient
import nest_asyncio

# Apply nested asyncio compatibility for Jupyter or similar environments
nest_asyncio.apply()

DEVICE_ADDRESS = "19:9F:19:C0:C2:42"  # Replace with your device's address

# UUIDs for the characteristics
TEMPERATURE_UUID = "2A6E"
HUMIDITY_UUID = "2A6F"
PRESSURE_UUID = "2A6D"
SOIL_MOISTURE_UUID = "2A70"

# Callback for disconnection
def handle_disconnection(client):
    print("Disconnected from central.")

async def main():
    client = BleakClient(DEVICE_ADDRESS, timeout=30.0)

    # Assign the disconnection callback
    client.set_disconnected_callback(handle_disconnection)

    try:
        await client.connect()
        if not client.is_connected:
            print("Failed to connect to device.")
            return

        print("Connected to device.")
        while True:
            try:
                # Read characteristic values
                temperature = await client.read_gatt_char(TEMPERATURE_UUID)
                humidity = await client.read_gatt_char(HUMIDITY_UUID)
                pressure = await client.read_gatt_char(PRESSURE_UUID)
                soil_moisture = await client.read_gatt_char(SOIL_MOISTURE_UUID)

                # Decode data (assuming IEEE 754 single-precision float, little-endian)
                import struct
                temperature = struct.unpack('<f', temperature)[0]
                humidity = struct.unpack('<f', humidity)[0]
                pressure = struct.unpack('<f', pressure)[0]
                soil_moisture = struct.unpack('<f', soil_moisture)[0]

                # Print the decoded values
                print(f"Temperature: {temperature:.2f} °C")
                print(f"Humidity: {humidity:.2f} %")
                print(f"Pressure: {pressure:.2f} hPa")
                print(f"Soil Moisture: {soil_moisture:.2f} %")
                print("----------------------------")

                await asyncio.sleep(1)
            except KeyboardInterrupt:
                print("Stopped by user.")
                break
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if client.is_connected:
            await client.disconnect()
        print("Program ended.")

# Use the event loop
loop = asyncio.get_event_loop()
loop.run_until_complete(main())


C:\Users\user\AppData\Local\Temp\ipykernel_552\2914232223.py:24: FutureWarning: This method will be removed future version, pass the callback to the BleakClient constructor instead.
  client.set_disconnected_callback(handle_disconnection)


Connected to device.
Temperature: 24.61 °C
Humidity: 53.58 %
Pressure: 102.16 hPa
Soil Moisture: 99.00 %
----------------------------
Temperature: 24.61 °C
Humidity: 53.60 %
Pressure: 102.16 hPa
Soil Moisture: 99.00 %
----------------------------
Temperature: 24.61 °C
Humidity: 53.58 %
Pressure: 102.16 hPa
Soil Moisture: 99.00 %
----------------------------
Temperature: 24.60 °C
Humidity: 53.51 %
Pressure: 102.16 hPa
Soil Moisture: 99.00 %
----------------------------


KeyboardInterrupt: 

Temperature: 24.59 °C
Humidity: 53.52 %
Pressure: 102.16 hPa
Soil Moisture: 99.00 %
----------------------------
Temperature: 24.59 °C
Humidity: 53.54 %
Pressure: 102.16 hPa
Soil Moisture: 99.00 %
----------------------------
